In [1]:
# default_exp benchmarking

In [2]:
#export
import matplotlib.pyplot as plt
import numpy as np


def plot_lines(protvals, log = True):
    """plot peptide profiles for a protein"""
    colors = plt.cm.tab20c(np.linspace(0,1,protvals.shape[0]))
    #cmap = plt.get_cmap("tab20c")
    idx = 0
    for row in protvals:
        if not log:
            row = 2**row
        plt.plot(row, c= colors[idx])
        idx+=1
    median_row = np.nanmedian(protvals, axis=0)
    print(median_row)
    plt.plot(median_row, c = 'black',linewidth =3 )
    plt.show()

def plot_points(protvals, log = True):
    colors = plt.cm.tab20c(np.linspace(0,1,protvals.shape[0]))
    #cmap = plt.get_cmap("tab20c")
    idx = 0
    for row in protvals:
        if not log:
            row = 2**row
        x_coord = list(range(len(row)))
        plt.scatter(x_coord,row, c= colors[idx])
        idx+=1

    plt.show()

In [3]:
#export
import pandas as pd
def get_tps_fps(result_df, prot2org_file, thresh = 0.05, fc_thresh = 0.3):
    annotated = annotate_dataframe(result_df, prot2org_file)
    condpairs = result_df["condpair"].drop_duplicates()


    for condpair in condpairs:
        annotated_condpair = annotated[annotated["condpair"]==condpair]
        num_tps = sum(annotated_condpair["TP"])
        num_fps = sum(annotated_condpair["FP"])
        annotated_fcfilt = annotated_condpair[annotated["log2fc"] >fc_thresh]
        num_regulated_prots = sum(annotated_fcfilt["fdr"]<thresh)
        num_true_positives = sum(annotated_fcfilt["TP"] &(annotated_fcfilt["fdr"]<0.05))
        num_false_positives = sum(annotated_fcfilt["FP"] &(annotated_fcfilt["fdr"]<0.05))
        fpr = num_false_positives/num_regulated_prots

        print(f'condpair {condpair}')
        print(f"total TPs {num_tps}")
        print(f"total FPs {num_fps}")
        print(f'regulated {num_regulated_prots}')
        print(f'false positives {num_false_positives}')
        print(f'true positives {num_true_positives}')
        print(f'regulated control {num_false_positives+num_true_positives}')
        print(f'FPR {fpr}')

        assert fpr < 0.06


def annotate_dataframe(result_df, prot2org_file):
    prot2org = pd.read_csv(prot2org_file, sep = "\t")
    prot2org["FP"] = (prot2org["organism"] == "Homo sapiens")
    prot2org["TP"] = (prot2org["organism"] == "Saccharomyces cerevisiae")
    prot2org = prot2org[(prot2org["FP"] | prot2org["TP"])]
    print(f"df size before {len(result_df.index)}")
    annotated = pd.merge(result_df, prot2org, how='inner', on = "protein")
    print(f"df size after {len(annotated.index)}")
    return annotated

In [4]:
#export
import pandas as pd
import matplotlib.pyplot as plt
def compare_to_reference(result_df, reference_file, condpair):#put in condpair as tuple
    result_df = result_df[result_df["condpair"]==condpair]

    ref_df = pd.read_csv(reference_file, sep = "\t")
    merged = pd.merge(result_df, ref_df, how='inner', on = "protein",suffixes = ["", "_ref"])
    ax_p = merged.plot.scatter(x='pval_ref',y='pval')
    plt.show()
    ax_fc = merged.plot.scatter(x='log2FC_ref',y='fc')
    plt.show()
    ax_fdr = merged.plot.scatter(x='fdr_ref',y='fdr')
    plt.show()

In [5]:
#export
import pandas as pd
import matplotlib.pyplot as plt
def compare_normalization(ref_normalization_file, norm1_df, norm2_df):
    ref_normed = pd.read_csv(ref_normalization_file, sep ="\t").set_index('peptide')

    merged = pd.merge(norm1_df, norm2_df, how='inner',  left_index = True, right_index = True)
    columns = merged.columns
    merged = pd.merge(ref_normed, merged, how='inner', left_index = True, right_index = True, suffixes = ["_ref", ""])

    for i in range(len(columns)):
        sample1 = columns[i]
        sample2 = sample1+"_ref"
        ax_p = merged.plot.scatter(x=sample1,y=sample2)
        plt.show()

In [6]:
#export
def print_nonref_hits(protein_ref, protein_df, peptide_ref, peptide_df, outdir):
    prots_nonref_df =  protein_df[~(protein_df["protein"].isin(protein_ref["protein"].to_list()))]#the tilde inverts the boolean vector
    peps_nonref_df = peptide_df[~(peptide_df["peptide"].isin(peptide_ref["peptide"].to_list()))]
    prots_nonref_df.to_csv(f"{outdir}/nonref_proteins.tsv", sep = "\t", index = False)
    peps_nonref_df.to_csv(f"{outdir}/nonref_peptides.tsv", sep = "\t", index = False)
    #display(peps_nonref_df)

In [7]:
#export
import pandas as pd
import functools


class ResultsTable():
    def __init__(self, input_file, input_name):
        self._input_file = input_file
        self._samples_c1 = None
        self._samples_c2 = None
        self.input_name = input_name
        self.protein_column = "protein"
        self.organism_column = "organism"
        self.log2fc_column = 'log2fc'
        self.mean_intensity_column = 'mean_intensity'
        self.formated_dataframe = None

    
    def get_proteins(self):
        return self.formated_dataframe[self.protein_column]

    def _add_mean_intensity_column(self):
        self._add_median_intensity_columns_for_each_condition()
        self.formated_dataframe[self.mean_intensity_column] = self.formated_dataframe[['median_intensity_c1', 'median_intensity_c2']].mean(axis = 1)

    
    def _add_median_intensity_columns_for_each_condition(self):
        self.formated_dataframe['median_intensity_c1'] = self.formated_dataframe[self._samples_c1].median(axis =1)
        self.formated_dataframe['median_intensity_c2'] = self.formated_dataframe[self._samples_c2].median(axis =1)

    def _add_log2fc_column(self):
        self.formated_dataframe[self.log2fc_column] = [np.log2(x[0]) - np.log2(x[1]) for x in zip(self.formated_dataframe['median_intensity_c1'], self.formated_dataframe['median_intensity_c2'])]




class ResultsTableDirectLFQ(ResultsTable):
    def __init__(self, input_file, input_name,  samples_c1, samples_c2):
        super().__init__(input_file, input_name)
        self._samples_c1 = samples_c1
        self._samples_c2 = samples_c2
        self._define_formatted_dataframe()
        
    def _define_formatted_dataframe(self):
        self._load_directlfq_output()
        self._add_median_intensity_columns_for_each_condition()
        self._add_log2fc_column()
        self._add_mean_intensity_column()
        
    def _load_directlfq_output(self):
        self.formated_dataframe =  pd.read_csv(self._input_file, sep = "\t")



class ResultsTableMaxQuant(ResultsTable):
    def __init__(self, input_file, input_name,  samples_c1, samples_c2):
        super().__init__(input_file, input_name)
        self._samples_c1 = samples_c1
        self._samples_c2 = samples_c2
        self._define_formatted_dataframe()

    
    def _define_formatted_dataframe(self):
        self._load_maxquant_output()
        self._add_median_intensity_columns_for_each_condition()
        self._add_log2fc_column()
        self._add_mean_intensity_column()

    def _load_maxquant_output(self):
        columns_to_use = self._samples_c1 + self._samples_c2 + ["id"]
        self.formated_dataframe = pd.read_csv(self._input_file, sep = "\t", usecols=columns_to_use)
        self.formated_dataframe = self.formated_dataframe.rename({'id':self.protein_column},axis=1)




In [9]:
#export
from abc import ABC, abstractmethod

class OrganismAnnotator(ABC):
    def __init__(self, mapping_file, protein_column = 'id', organism_column = 'Species'):
        self._mapping_file = mapping_file
        self._protein_column = protein_column
        self._organism_column = organism_column
        self._protein_organism_mapping_df = self._load_reduce_mapping_dataframe()
        super().__init__()

    def annotate_table_with_organism(self, results_table):

        self.__add_organism_column_to_results_table(results_table)
        self.__filter_non_matching_proteins(results_table)
        

    def save_protein_organism_map(self, outfile):
        self._protein_organism_mapping_df.to_csv(outfile, sep = "\t", index = None)
    
    @abstractmethod
    def _load_reduce_mapping_dataframe(self):
        pass
    
    def _filter_double_mapping_organism(self, protein2organism_df):
        protein2organism_df = protein2organism_df[[";" not in x for x in protein2organism_df[self._organism_column].astype('str')]] #a semicolon seperates different organism entries
        return protein2organism_df

    def __add_organism_column_to_results_table(self, results_table):
        protein2organism_dict = self.__get_protein2organism_dict()
        proteins_resultstable = results_table.formated_dataframe[results_table.protein_column].astype('str')
        results_table.formated_dataframe[results_table.organism_column] = [protein2organism_dict.get(x) for x in proteins_resultstable]

    def __get_protein2organism_dict(self):
        protein2organism = dict(zip(self._protein_organism_mapping_df[self._protein_column].astype('str'), self._protein_organism_mapping_df[self._organism_column]))
        return protein2organism
    
    @staticmethod
    def __filter_non_matching_proteins(results_table):
        list_indicating_if_protein_matches = [x is not None for x in results_table.formated_dataframe[results_table.organism_column]]
        results_table.formated_dataframe = results_table.formated_dataframe[list_indicating_if_protein_matches]
        


In [10]:
#export
class OrganismAnnotatorMaxQuant(OrganismAnnotator):
    def __init__(self, mapping_file, protein_column = 'id', organism_column = 'Species'):
        super().__init__(mapping_file=mapping_file, protein_column= protein_column, organism_column= organism_column)
    
    def _load_reduce_mapping_dataframe(self):
        mapping_df = pd.read_csv(self._mapping_file, sep = "\t", usecols=[self._protein_column, self._organism_column], encoding='latin1').drop_duplicates()
        mapping_df = self._filter_double_mapping_organism(mapping_df)
        return mapping_df


In [ ]:
#export
class OrganismAnnotatorSpectronaut(OrganismAnnotator):
    def __init__(self, mapping_file,protein_column="PG.ProteinGroups", organism_column="PG.Organisms"):
        super().__init__(mapping_file=mapping_file, protein_column= protein_column, organism_column= organism_column)
    
    def _load_reduce_mapping_dataframe(self):
        mapping_df = pd.read_csv(self._mapping_file, sep = "\t", usecols=[self._protein_column, self._organism_column], encoding='latin1').drop_duplicates()
        mapping_df = self._filter_double_mapping_organism(mapping_df)
        return mapping_df


In [11]:
#export
class OrganismAnnotatorDIANN(OrganismAnnotator):
    def __init__(self, mapping_file, protein_column = 'Protein.Group', organism_column = 'Protein.Names'):
        super().__init__(mapping_file=mapping_file, protein_column= protein_column, organism_column= organism_column)
    
    def _load_reduce_mapping_dataframe(self):
        mapping_df = pd.read_csv(self._mapping_file, sep = "\t", usecols=[self._protein_column, self._organism_column], encoding='latin1').drop_duplicates()
        mapping_df[self._organism_column] = [self.__get_organism_from_protein_name(x) for x in mapping_df[self._organism_column]]
        mapping_df = mapping_df.drop_duplicates()
        return mapping_df
    
    @staticmethod
    def __get_organism_from_protein_name(protein_name):
        split_name = protein_name.split("_")
        if len(split_name) <2:
            return "None"
        else:
            return split_name[-1]

In [12]:
#export
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


class PlotConfig():
    def __init__(self, x_axisboundaries = None, y_axisboundaries = None, colormaps = None):
        self.x_axisboundaries = x_axisboundaries
        self.y_axisboundaries = y_axisboundaries
        self._colormaps = colormaps

class MultiOrganismIntensityFCPlotter():
    def __init__(self, ax, resultstable_w_ratios, plotconfig = PlotConfig()):
        self.ax = ax
        self._resultstable_w_ratios = resultstable_w_ratios
        self._organism_column = resultstable_w_ratios.organism_column
        self._log2fc_column = resultstable_w_ratios.log2fc_column
        self._mean_intensity_column = resultstable_w_ratios.mean_intensity_column
        self._plotconfig = plotconfig
        
        self._all_organisms = self._get_all_organisms_used()
        self._title = self._get_title()
        self._scatter_per_organism()
        

    def _get_all_organisms_used(self):
        return sorted(list(set(self._resultstable_w_ratios.formated_dataframe[self._organism_column].astype('str'))))
    
    def _get_title(self):
        title = ""
        for organism in self._all_organisms:
            subtable_organism = self._get_organism_subtable(organism)
            title += self._extend_title(organism,subtable_organism)
        return title

    def _scatter_per_organism(self):
        complete_table = self._resultstable_w_ratios.formated_dataframe.copy()
        complete_table[self._mean_intensity_column] = np.log2(complete_table[self._mean_intensity_column])
        sns.scatterplot(data= complete_table, x = self._log2fc_column, y=self._mean_intensity_column, hue=self._organism_column, alpha=0.15, ax=self.ax, hue_order=self._all_organisms)
        self.ax.set_title(self._title)
        self.ax.set_xlim(self._plotconfig.x_axisboundaries)

    def _get_organism_subtable(self, organism):
        complete_table = self._resultstable_w_ratios.formated_dataframe
        return complete_table[complete_table[self._organism_column] == organism]
    
    def _extend_title(self, organism, subtable_organism):
        fcs = subtable_organism[self._log2fc_column].to_numpy()
        fcs = fcs[np.isfinite(fcs)]
        median_fc = np.nanmedian(fcs)
        std_fc = np.nanstd(fcs)
        num_ratios = sum(~np.isnan(fcs))
        return f"{organism} num:{num_ratios} median_FC:{median_fc:.2} STD:{std_fc:.2}\n"
 



### Obtain CV distribution from ProteinIntensityTable

In [2]:
#export
import pandas as pd
import directlfq.utils as lfq_utils

class ResultsTableBiological():
    def __init__(self, results_file, samplemap, name):
        self._results_file = results_file
        self._samplemap = samplemap
        self._name = name

        self.results_df = None
        self.cond2samples = {}

        self._load_results_table()
        self._load_cond2samples()

    def _load_results_table(self):
        self.results_df = pd.read_csv(self._results_file, sep = "\t")
    
    def _load_cond2samples(self):
        samplemap_df = lfq_utils.load_samplemap(self._samplemap)
        for sample, cond in zip(samplemap_df["sample"], samplemap_df["condition"]):
            self.cond2samples[cond] = self.cond2samples.get(cond, []) + [sample]
    


In [21]:
#export
import numpy as np
class CVcalculator():
    def __init__(self, resultstable_biological):
        self._results_table = resultstable_biological
        self.cvs = []
        self._calculate_cvs()

    def _calculate_cvs(self):
        for samples in self._results_table.cond2samples.values():
            self._add_protein_cvs_for_condition(samples)
    
    def _add_protein_cvs_for_condition(self, samples):
        subtable = self._results_table.results_df[samples]
        protein_cvs = subtable.apply(self._cv_function,axis = 1)
        protein_cvs = [x for x in protein_cvs if not np.isnan(x)]
        self.cvs += list(protein_cvs)

    @staticmethod
    def _cv_function(x):
        if sum(~np.isnan(x)) <2:
            return np.nan
        return np.nanstd(x, ddof=1,) / np.nanmean(x) ##ddof ensures that the sample mean std estimate is used

### Test CV distribution estimation

In [22]:
#hide

class ResTableTest(ResultsTableBiological):
    def __init__(self):
        pass
def test_cv_estimation():
    restable_test = ResTableTest()
    restable_test.cond2samples = {"cond" : ['A', 'B'], "othercond" : ['A', 'C']}
    restable_test.results_df = pd.DataFrame({'A' : [1, 2, 3], 'B' : [1, 2, 3], 'C' : [3, 2, 1]})
    calced_cvs = CVcalculator(restable_test).cvs
    assert np.allclose([0.0, 0.0, 0.0, 0.7071067811865476, 0.0, 0.7071067811865476], calced_cvs)

test_cv_estimation()

In [23]:
input_file = "test_data/system_tests/lfqbench/DIANN/report.tsv.diann_fragion_isotopes_raw.aq_reformat.tsv.protein_intensities.tsv"
samplemap_file = "test_data/system_tests/lfqbench/DIANN/samplemap.tsv"

biol_res = ResultsTableBiological(results_file=input_file, samplemap=samplemap_file, name="diann")
biol_res.cond2samples
biol_res.results_df

calced_cvs = CVcalculator(biol_res)

/var/folders/wb/j08kzzbd2hd8838tn3ll0jfm0000gn/T/ipykernel_1320/1206190671.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return np.nanstd(x, ddof=1,) / np.nanmean(x) ##ddof ensures that the sample mean std estimate is used
/var/folders/wb/j08kzzbd2hd8838tn3ll0jfm0000gn/T/ipykernel_1320/1206190671.py:23: RuntimeWarning: invalid value encountered in double_scalars
  return np.nanstd(x, ddof=1,) / np.nanmean(x) ##ddof ensures that the sample mean std estimate is used


### Create input dataframes with scalable number of samples

In [14]:
#export
import pandas as pd
import directlfq.lfq_manager as lfqmgr
import directlfq.normalization as lfqnorm
import directlfq.protein_intensity_estimation as lfq_protein_estimation
import time
import math

class InputDFCreator():
    def __init__(self, template_df, desired_number_of_samples):
        self._template_df = template_df
        self._desired_number_of_samples = desired_number_of_samples
        self._column_names = list(self._template_df.columns)
        self._header_list = [] 
        self._create_list_of_df_headers()
        
        self.input_df = None

        self._create_input_dataframe()

    def _create_input_dataframe(self):

        template_dataframe_dict = self._template_df.to_dict(orient = 'list')
        input_dataframe_dict = {x : template_dataframe_dict.get(self._get_original_column_name(x))  for x in self._header_list}
        self.input_df = pd.DataFrame(input_dataframe_dict, index=self._template_df.index)

    def _create_list_of_df_headers(self):
        self._append_replicate_column_names_to_headerlist()
        self._append_remaining_column_names_to_headerlist()
    
    
    def _append_replicate_column_names_to_headerlist(self):
        num_samples_in_df = len(self._column_names)
        num_replicates_of_column_names = math.floor(self._desired_number_of_samples/num_samples_in_df)
        for idx in range(num_replicates_of_column_names):
            self._header_list += [f"{x}_AND_{idx}" for x in self._column_names]

    def _append_remaining_column_names_to_headerlist(self):
        num_samples_in_df = len(self._column_names)
        num_remaining_samples = self._desired_number_of_samples%num_samples_in_df
        self._header_list +=[f"{self._column_names[x]}_AND_remainder" for x in range(num_remaining_samples)]

    @staticmethod
    def _get_original_column_name(new_column_name):
        return new_column_name.split("_AND_")[0]

### Test input df creation

In [16]:
#hide
import pandas as pd
import numpy as np

class TemplateDFCreator():
    def __init__(self):
        self.template_df = None
        self._template_df_location = "test_data/unit_tests/protein_normalization/example_proteins.tsv"
        self._create_template_df()

    def _create_template_df(self):
        self.template_df = pd.read_csv(self._template_df_location, index_col= ["protein", "ion"], sep = "\t")


def test_that_shape_is_as_expected(num_samples):
    template_df = TemplateDFCreator().template_df
    size_adjusted_df = InputDFCreator(template_df=template_df, desired_number_of_samples= num_samples).input_df
    assert len(size_adjusted_df.columns) == num_samples
    assert len(size_adjusted_df.index) == len(template_df.index)
    print("performed_checks")


test_that_shape_is_as_expected(1)
test_that_shape_is_as_expected(100)
test_that_shape_is_as_expected(10000)

def test_that_values_are_as_expected(num_samples):
    template_df = TemplateDFCreator().template_df
    size_adjusted_df = InputDFCreator(template_df=template_df, desired_number_of_samples= num_samples).input_df
    assert np.allclose(template_df.loc[:,"BoxCar_02-01_2"], size_adjusted_df.loc[:, "BoxCar_02-01_2_AND_remainder"])


test_that_values_are_as_expected(5)
test_that_values_are_as_expected(100)



performed_checks
performed_checks
performed_checks


In [ ]:
#export        

class TimedLFQRun():
    def __init__(self, formatted_df, name):
        self.name = name
        self.runtime_info = RuntimeInfo()
        self.num_samples = len(formatted_df.columns)
        self._formatted_df = formatted_df

    def run_from_formatted_df(self):
        self.runtime_info._start_samplenorm = time.time()
        input_df_normed = lfqnorm.SampleNormalizationManager(self._formatted_df).complete_dataframe
        self.runtime_info._end_samplenorm = time.time()
        self.runtime_info._start_protein_norm = time.time()
        lfq_protein_estimation.estimate_protein_intensities(input_df_normed,min_nonan=2,maximum_df_length=100)
        self.runtime_info._end_protein_norm = time.time()
        self.runtime_info.calculate_runtimes()

class RuntimeInfo():
    def __init__(self):
        self._start_samplenorm= None
        self._end_samplenorm = None
        self._start_protein_norm = None
        self._end_protein_norm = None

        self.overall_runtime = None
        self.runtime_samplenorm = None
        self.runtime_protein_norm = None

    def calculate_runtimes(self):
        self.overall_runtime = (self._end_protein_norm - self._start_samplenorm)/60
        self.runtime_samplenorm = (self._end_samplenorm - self._start_samplenorm)/60
        self.runtime_protein_norm = (self._end_protein_norm - self._start_protein_norm)/60


In [4]:
#export
import pandas as pd
import directlfq.utils as lfq_utils

class LFQTimer():
    def __init__(self, template_file, list_of_samplenumbers_to_check, name):
        self._template_file = template_file
        self._samplenumbers_to_check = list_of_samplenumbers_to_check
        self._name = name
        self._template_df = None
        self._read_template_file()
        self.timed_lfq_runs = []
        self._iterate_through_sizes()
    
    def _read_template_file(self):
        self._template_df = pd.read_csv(self._template_file, sep = "\t")
        self._template_df = lfq_utils.index_and_log_transform_input_df(self._template_df)

    def _iterate_through_sizes(self):
        for samplenumber in self._samplenumbers_to_check:
            formatted_df = InputDFCreator(self._template_df, desired_number_of_samples=samplenumber).input_df
            self.timed_lfq_runs.append(TimedLFQRun(formatted_df,self._name).run_from_formatted_df())


### Learning Tests

In [15]:
import seaborn as sns
import numpy as np


def check_taking_mean_of_pandas_columns():
    df = sns.load_dataset('iris')
    display(df)
    df.iloc[3:6, 1] = np.nan
    display(df)
    df['mean_sepal_width_petal_length'] = df[["sepal_width", "petal_length"]].median(axis = 1, skipna = True)
    assert df.loc[3, 'mean_sepal_width_petal_length'] == df.loc[3, 'petal_length']


check_taking_mean_of_pandas_columns()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,NaN,1.5,0.2,setosa
4,5.0,NaN,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
def check_cv_calculation_along_a_dataframe_axis():
        
    df_test = sns.load_dataset("iris").drop(labels = "species", axis = 1)
    cv_function = lambda x: np.std(x, ddof=1,) / np.mean(x)
    cv_results = df_test.apply(cv_function, axis=1)
    display(df_test)
    display(cv_results)

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


0      0.854686
1      0.857663
2      0.849999
3      0.813720
4      0.845642
         ...   
145    0.470128
146    0.528880
147    0.490238
148    0.414200
149    0.476999
Length: 150, dtype: float64